# API 활용하여 Covid19 데이터 불러오기

In [46]:
import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd

## API 사용법

### 공공데이터 포털 API 사용하기

In [47]:
serviceKey = 'MWv5yo0tJfyukfZUGm7gU3e8hb16XH1%2FSuvW89vS6B9%2BG%2FcrxFnQxRMx6JiGGMVqMQUp4HOqov60IPOU0yc%2BXQ%3D%3D' # 공공데이터포털에서 발급받은 서비스키
params = {'ServiceKey':parse.unquote(serviceKey), # 서비스키(필수)
          'startCreateDt':20210901, # 데이터 생성일 시작범위(선택)
          'endCreateDt':20210916 # 데이터 생성일 종료범위(선택)
         }
# 서비스URL
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'

res = requests.get(url, params=params)
soup = bs(res.text, 'lxml')

# 결과 출력
print(soup)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>13</deathcnt><defcnt>5970</defcnt><gubun>검역</gubun><gubuncn>隔離區</gubuncn><gubunen>Lazaretto</gubunen><incdec>7</incdec><isolclearcnt>5707</isolclearcnt><isolingcnt>250</isolingcnt><localocccnt>0</localocccnt><overflowcnt>7</overflowcnt><qurrate>-</qurrate><seq>12150</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item><item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>2</deathcnt><defcnt>2754</defcnt><gubun>제주</gubun><gubuncn>济州</gubuncn><gubunen>Jeju</gubunen><incdec>11</incdec><isolclearcnt>2653</isolclearcnt><isolingcnt>99</isolingcnt><localocccnt>11</localocccnt><overflowcnt>0</overflowcnt><qurrate>406.59</qurrate><seq>12149</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item><item><createdt>2021-09-16 09:54:50.604

### XML 형태를 DataFrame 형태로 변경하기  
원하는 정보는 item 태그 내부에 존재

In [48]:
# BeautifulSoup을 사용해 item 태그만 리스트 형태로 가져옵니다.
items = soup.find_all('item')
print(items)

[<item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>13</deathcnt><defcnt>5970</defcnt><gubun>검역</gubun><gubuncn>隔離區</gubuncn><gubunen>Lazaretto</gubunen><incdec>7</incdec><isolclearcnt>5707</isolclearcnt><isolingcnt>250</isolingcnt><localocccnt>0</localocccnt><overflowcnt>7</overflowcnt><qurrate>-</qurrate><seq>12150</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item>, <item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>2</deathcnt><defcnt>2754</defcnt><gubun>제주</gubun><gubuncn>济州</gubuncn><gubunen>Jeju</gubunen><incdec>11</incdec><isolclearcnt>2653</isolclearcnt><isolingcnt>99</isolingcnt><localocccnt>11</localocccnt><overflowcnt>0</overflowcnt><qurrate>406.59</qurrate><seq>12149</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item>, <item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>36</deathcnt><defcnt>10768</defcnt><gubun>경남</gubun><gubuncn>庆南</gubuncn><gubunen>Gyeongsangnam-do</gubunen><incdec>33</incdec><isolclearcnt

In [49]:
# 리스트 원소 확인
for x in items:
    print(x)
    print()

<item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>13</deathcnt><defcnt>5970</defcnt><gubun>검역</gubun><gubuncn>隔離區</gubuncn><gubunen>Lazaretto</gubunen><incdec>7</incdec><isolclearcnt>5707</isolclearcnt><isolingcnt>250</isolingcnt><localocccnt>0</localocccnt><overflowcnt>7</overflowcnt><qurrate>-</qurrate><seq>12150</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item>

<item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>2</deathcnt><defcnt>2754</defcnt><gubun>제주</gubun><gubuncn>济州</gubuncn><gubunen>Jeju</gubunen><incdec>11</incdec><isolclearcnt>2653</isolclearcnt><isolingcnt>99</isolingcnt><localocccnt>11</localocccnt><overflowcnt>0</overflowcnt><qurrate>406.59</qurrate><seq>12149</seq><stdday>2021년 09월 16일 00시</stdday><updatedt>null</updatedt></item>

<item><createdt>2021-09-16 09:54:50.604</createdt><deathcnt>36</deathcnt><defcnt>10768</defcnt><gubun>경남</gubun><gubuncn>庆南</gubuncn><gubunen>Gyeongsangnam-do</gubunen><incdec>33</incdec><isolclearcnt>

In [53]:
# xml을 dictionary로 변경
response = {
            'createdt': '등록일시분초',
            'deathcnt': '사망자 수',
            'defcnt': '확진자 수',
            'gubun': '시도명(한글)',
            'gubuncn': '시도명(중국어)',
            'gubunen': '시도명(영어)',
            'incdec': '전일대비 증감 수',
            'isolclearcnt': '격리 해제 수',
            'isolingcnt': '격리중 환자 수',
            'localocccnt': '지역발생 수',
            'overflowcnt': '해외유입 수',
            'qurrate': '10만명당 발생률',
            'seq': '게시글번호',
            'stdday': '기준일시',
            'updatedt': '수정일시분초'
            }

#### XML 태그

In [54]:
lst = []
for y in items:
    l = {}
    for x in y:
        l[x.name] = x.text
    lst.append(l)
lst

[{'createdt': '2021-09-16 09:54:50.604',
  'deathcnt': '13',
  'defcnt': '5970',
  'gubun': '검역',
  'gubuncn': '隔離區',
  'gubunen': 'Lazaretto',
  'incdec': '7',
  'isolclearcnt': '5707',
  'isolingcnt': '250',
  'localocccnt': '0',
  'overflowcnt': '7',
  'qurrate': '-',
  'seq': '12150',
  'stdday': '2021년 09월 16일 00시',
  'updatedt': 'null'},
 {'createdt': '2021-09-16 09:54:50.604',
  'deathcnt': '2',
  'defcnt': '2754',
  'gubun': '제주',
  'gubuncn': '济州',
  'gubunen': 'Jeju',
  'incdec': '11',
  'isolclearcnt': '2653',
  'isolingcnt': '99',
  'localocccnt': '11',
  'overflowcnt': '0',
  'qurrate': '406.59',
  'seq': '12149',
  'stdday': '2021년 09월 16일 00시',
  'updatedt': 'null'},
 {'createdt': '2021-09-16 09:54:50.604',
  'deathcnt': '36',
  'defcnt': '10768',
  'gubun': '경남',
  'gubuncn': '庆南',
  'gubunen': 'Gyeongsangnam-do',
  'incdec': '33',
  'isolclearcnt': '10194',
  'isolingcnt': '538',
  'localocccnt': '32',
  'overflowcnt': '1',
  'qurrate': '321.39',
  'seq': '12148',
  's

In [55]:
df = pd.DataFrame(lst)
df.columns = df.columns.map(response)
df

,NaN,사망자 수,확진자 수,시도명(한글),NaN,NaN,전일대비 증감 수,격리 해제 수,격리중 환자 수,지역발생 수,해외유입 수,10만명당 발생률,NaN,기준일시,NaN
0,2021-09-16 09:54:50.604,13,5970,검역,隔離區,Lazaretto,7,5707,250,0,7,-,12150,2021년 09월 16일 00시,null
1,2021-09-16 09:54:50.604,2,2754,제주,济州,Jeju,11,2653,99,11,0,406.59,12149,2021년 09월 16일 00시,null
2,2021-09-16 09:54:50.604,36,10768,경남,庆南,Gyeongsangnam-do,33,10194,538,32,1,321.39,12148,2021년 09월 16일 00시,null
3,2021-09-16 09:54:50.604,89,7632,경북,庆北,Gyeongsangbuk-do,33,7195,348,31,2,287.9,12147,2021년 09월 16일 00시,null
4,2021-09-16 09:54:50.603,20,2850,전남,全南,Jeollanam-do,8,2651,179,8,0,153.49,12146,2021년 09월 16일 00시,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2021-09-06 09:00:00.0,2327,261774,합계,合?,Total,1375,233695,25752,1351,24,505.08,11324,2021년 09월 06일 00시,2021-09-14 15:54:32.542
313,2021-09-06 09:00:00.0,593,82654,서울,首?,Seoul,464,72973,9088,462,2,854.90,11323,2021년 09월 06일 00시,2021-09-14 15:54:14.196
314,2021-09-06 09:00:00.0,759,74365,경기,京畿,Gyeonggi-do,400,65468,8138,398,2,553.85,11315,2021년 09월 06일 00시,2021-09-14 10:23:13.361
315,2021-09-01 09:00:00.0,2292,253439,합계,合?,Total,2024,0,251147,1991,33,488.99,11229,2021년 09월 01일 00시,2021-09-14 15:49:06.857


In [45]:
df.to_excel('covid19_2109.xlsx', index = False)
# df.to_excel('covid19_2108.xlsx')
# 21년 9월 8월(일부분) 저장

#### xmltodict 라이브러리 사용하기

In [8]:
import xmltodict
xmltodict.parse(str(items[0]))['item']

OrderedDict([('createdt', '2021-09-16 09:54:50.604'),
             ('deathcnt', '13'),
             ('defcnt', '5970'),
             ('gubun', '검역'),
             ('gubuncn', '隔離區'),
             ('gubunen', 'Lazaretto'),
             ('incdec', '7'),
             ('isolclearcnt', '5707'),
             ('isolingcnt', '250'),
             ('localocccnt', '0'),
             ('overflowcnt', '7'),
             ('qurrate', '-'),
             ('seq', '12150'),
             ('stdday', '2021년 09월 16일 00시'),
             ('updatedt', 'null')])

In [9]:
import xmltodict
dict(xmltodict.parse(str(items[0]))['item'])

{'createdt': '2021-09-16 09:54:50.604',
 'deathcnt': '13',
 'defcnt': '5970',
 'gubun': '검역',
 'gubuncn': '隔離區',
 'gubunen': 'Lazaretto',
 'incdec': '7',
 'isolclearcnt': '5707',
 'isolingcnt': '250',
 'localocccnt': '0',
 'overflowcnt': '7',
 'qurrate': '-',
 'seq': '12150',
 'stdday': '2021년 09월 16일 00시',
 'updatedt': 'null'}

In [10]:
dic2 = dict(covid19=[])
for item in items:
    dic2['covid19'].append(dict(xmltodict.parse(str(item))['item']))

# DataFrame으로 변환
df = pd.DataFrame(dic2['covid19'])
df.columns = df.columns.map(response) # column명에 map함수를 적용하여 한글로 변환
df

,등록일시분초,사망자 수,확진자 수,시도명(한글),시도명(중국어),시도명(영어),전일대비 증감 수,격리 해제 수,격리중 환자 수,지역발생 수,해외유입 수,10만명당 발생률,게시글번호,기준일시,수정일시분초
0,2021-09-16 09:54:50.604,13,5970,검역,隔離區,Lazaretto,7,5707,250,0,7,-,12150,2021년 09월 16일 00시,null
1,2021-09-16 09:54:50.604,2,2754,제주,济州,Jeju,11,2653,99,11,0,406.59,12149,2021년 09월 16일 00시,null
2,2021-09-16 09:54:50.604,36,10768,경남,庆南,Gyeongsangnam-do,33,10194,538,32,1,321.39,12148,2021년 09월 16일 00시,null
3,2021-09-16 09:54:50.604,89,7632,경북,庆北,Gyeongsangbuk-do,33,7195,348,31,2,287.9,12147,2021년 09월 16일 00시,null
4,2021-09-16 09:54:50.603,20,2850,전남,全南,Jeollanam-do,8,2651,179,8,0,153.49,12146,2021년 09월 16일 00시,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2021-08-04 10:08:57.459,41,3262,울산,蔚山,Ulsan,15,3076,145,14,1,287.14,11225,2021년 08월 04일 00시,null
829,2021-08-04 10:08:57.459,30,4459,대전,大田,Daejeon,59,3308,1121,59,0,304.60,11224,2021년 08월 04일 00시,null
830,2021-08-04 10:08:57.459,24,3464,광주,光州,Gwangju,16,3225,215,15,1,238.89,11223,2021년 08월 04일 00시,null
831,2021-08-04 10:08:57.458,66,9342,인천,仁川,Incheon,83,8132,1144,83,0,317.45,11222,2021년 08월 04일 00시,2021-08-04 15:47:36.84
